In [37]:
import os
import sqlite3
import csv

DB = 'db'
TABLE = 'baseball'

path = 'data'
fn = 'py_baseball_data.csv'
file_path = os.path.join( path, fn)
file_path

'data\\py_baseball_data.csv'

In [38]:
def create_database(file_path, db, table):
    """
    Creates a database w/ table loaded with file data.
    
    :param str filepath:  Fully-specified path/to/file
    :param str db:  Name of database to be created
    :param str table: Name of the database table to be created
    :return: None   
    """
    
    # connect to the database
    conn = sqlite3.connect(db)
    curs = conn.cursor()

    # Set up a data table
    cmd = "DROP TABLE IF EXISTS {}".format(table)
    curs.execute(cmd)

    cmd = """CREATE TABLE {}
                         (year   INT, 
                          month  INT, 
                          day    INT, 
                          last   TEXT(20), 
                          first  TEXT(20), 
                          team   TEXT(30), 
                          league TEXT(2)
                          )""".format(table)
    curs.execute(cmd)
    conn.commit()

    # Capture the column names
    cmd = f"SELECT * FROM {table}"
    curs.execute(cmd)

    cols = []
    for name in curs.description:
        cols.append(name[0])
    cols = tuple(cols)

    # open the file and read it, dropping each row into the data table
    with open(file_path, 'r') as data_file:
        
        # Open the file with the reader and skip the header row
        reader = csv.reader(data_file)
        next(reader)  
        
        # Iterate through the data, inserting it into the db.
        for data in reader:
            # create tuples for our data and column values
            cmd = f"INSERT INTO {table} {cols} VALUES (?, ?, ?, ?, ?, ?, ?) "
            curs.execute(cmd, tuple(data))
            
    conn.commit()
    conn.close()

In [39]:
def get_nh_results_nl_after_june(db, table):
    
    """
    Queries a database.
    
    :param str db:  Name of database to be queried
    :param str table: Name of the database table to be queried
    :return list results:  Tuples of results
    
    """    
    "dig out the ho-hitters after June by NL teams"
    conn = sqlite3.connect(db)
    curs = conn.cursor()
    cmd = "SELECT * FROM {} WHERE league='NL' AND month>6".format(table)
    curs.execute(cmd)

    results = curs.fetchall()
    return results

In [40]:
# Execute our creation and query routines

create_database(file_path, DB, TABLE)
results = get_nh_results_nl_after_june(DB, TABLE)

print(results)

[(2014, 9, 28, 'Zimmermann', ' Jordan', 'Washington', 'NL'), (2015, 7, 25, 'Hamels', 'Cole', 'Philadelphia', 'NL'), (2015, 8, 30, 'Arrieta', ' Jake', 'Chicago', 'NL'), (2015, 10, 3, 'Scherzer', ' Max', 'Washington', 'NL')]
